# Атака на широковещательный вариант RSA (RSA Broadcast attack)
Представьте, что вы используете открытую экспоненту $e=3$ (на так давно она была достаточно популярна), но вы защифроваываете только сообщения, которые переаолняют модуль $N$ при возведении в третью степень, так что нельзя просто взять и извлечь корень третьей степени. Правда ли это безопасно? Есть довольно простой сценарий, который наглядно проказывает проблему небольшой открытой экспоненты.
Представьте, что пользователь использует RSA, чтобы посылать сообщения на несколько серверов (все с экспонентой 3, но разными модулями $N$). Пусть таких серверов 3. Одно и то же сообщение шифруется 3 раза: с ключами первого, второго и третьего серверов.
$$C_1=M^{3} mod N_1$$
$$C_2=M^{3} mod N_2$$
$$C_3=M^{3} mod N_3$$
Почти невозможно дешифровать каждый из шифротекстов $C_{i}$ по одинчке, но с тремя Марвин(или Мэллори, как вам больше нравится) можно решить эту задачу.
## Китайская теорема об остатках
Если известны остатки от Евклидова деления целого числа $n$ на несколько целых чисел, то можно определить уникальный остаток от деления $n$ на произведение этих целых чисел, при условии, что все делители (модули, по которым брали остатки) попарно простые. 

Как мы можем использовать эту теорему? Во-первых, нам необходимо проверить, что все делители  ($N_1, N_2, N_3$) взаимно (попарно) простые. Но, если вдруг они не взаимно простые, то наибольший общий делитель двух из них больше единицы и либо они равны (и мы не можем их испольовать), либо они имеют в составе одно и то же простое число -их $НОД$. В последнем случае мы и так можем расшифровать сообщение. Так что будем считать, что они попарно взаимнопростые. Это значит, что существует такой $X$, что:
$$ X\lt N_1 N_2 N_3$$
$$ X = C_1 mod N_1 $$
$$ X = C_2 mod N_2 $$
$$ X = C_3 mod N_3 $$
и такой $X$ уникален.

Давайте рассмотрим $C=M^{3}$. Так как $M\lt N_1$ и $M\lt N_2$ и $M\lt N_3$, то $C=M^3\lt N_1N_2N_3$. А ещё:
$$ C = C_1 mod N_1 $$
$$ C = C_2 mod N_2 $$
$$ C = C_3 mod N_3 $$

Так что используя критайскую теорему об остатках можно найти $C$ и всё, что останется сделать - это извлечь кубический корень.
## Как получить C?

Пусть $N_i, i=1,k$ - модули, а $c_i, i=1,k$ - остатки по делению. $N=N_1N_2...N_k$, а $M_i=N/N_i$
Тогда $C=(\sum_{i=0}^{k}C_iM_i(M_i^{-1}\space mod\space N_i))\space mod\space N$

Воспользуйтесь выражением и вытащите флаг из трёх зашифрованных сообщений, полученных с сервера. Удачи!

In [1]:
import socket
import re
from Crypto.Util.number import inverse,long_to_bytes,bytes_to_long
class VulnServerClient:
    def __init__(self,show=True):
        """Initialization, connecting to server"""
        self.s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.s.connect(('cryptotraining.zone',1339))
        if show:
            print (self.recv_until().decode())
    def recv_until(self,symb=b'\n>'):
        """Receive messages from server, by default till new prompt"""
        data=b''
        while True:
            
            data+=self.s.recv(1)
            if data[-len(symb):]==symb:
                break
        return data
    def get_public_keys(self,show=True):
        """Receive public keys from the server"""
        self.s.sendall('public\n'.encode())
        response=self.recv_until().decode()
        if show:
            print (response)
        e1=int(re.search('(?<=e1: )\d+',response).group(0))
        N1=int(re.search('(?<=N1: )\d+',response).group(0))
        e2=int(re.search('(?<=e2: )\d+',response).group(0))
        N2=int(re.search('(?<=N2: )\d+',response).group(0))
        e3=int(re.search('(?<=e3: )\d+',response).group(0))
        N3=int(re.search('(?<=N3: )\d+',response).group(0))
       
        return [(e1,N1),(e2,N2),(e3,N3)]
    
    def get_ciphertexts(self,show=True):
        """Receive ciphertexts from the server"""
        self.s.sendall('ciphertext\n'.encode())
        response=self.recv_until().decode()
        if show:
            print (response)
        c1=bytes_to_long(bytes.fromhex(re.search('(?<=ciphertext1: )[0-9a-f]+',response).group(0)))
        c2=bytes_to_long(bytes.fromhex(re.search('(?<=ciphertext2: )[0-9a-f]+',response).group(0)))
        c3=bytes_to_long(bytes.fromhex(re.search('(?<=ciphertext3: )[0-9a-f]+',response).group(0)))
        return (c1,c2,c3)
    
    def __del__(self):
        self.s.close()

In [2]:
vs=VulnServerClient()
pk_list=vs.get_public_keys()
(c1,c2,c3)=vs.get_ciphertexts()

Welcome to RSA broadcast task
Available commands:
help - print this help
public - show public keys
ciphertext - show ciphertexts 
quit - quit
>
e1: 3
N1: 20287982006618431876793244706487063574769448388426702838915722457901061849764724362603953179532539827554365329323483013276648891796378507103593000936891322846098718727133325953848182431476448546554772557085987908949429403596359635314342612889908898272272322173341141651567301687427226491229442385493785765699715986462483124423171652756203919879715705590771525305446788322512844427648822922682205388423707896633544989180321378196798302096862401020103125458117084856441433418990681274327810046291890889882496621395403208554291777227706389355365678885157011052465893070731243360783878242460400701935579682716345964783581
e2: 3
N2: 19461826656993775602233892694656909792660373232000384211810816270789889052589469816096210883392037679550505993511270364500797371312576866213808413522877454484246665882296260574836114845774508160454919015698629693496502

In [4]:
pk_list

[(3,
  20287982006618431876793244706487063574769448388426702838915722457901061849764724362603953179532539827554365329323483013276648891796378507103593000936891322846098718727133325953848182431476448546554772557085987908949429403596359635314342612889908898272272322173341141651567301687427226491229442385493785765699715986462483124423171652756203919879715705590771525305446788322512844427648822922682205388423707896633544989180321378196798302096862401020103125458117084856441433418990681274327810046291890889882496621395403208554291777227706389355365678885157011052465893070731243360783878242460400701935579682716345964783581),
 (3,
  194618266569937756022338926946569097926603732320003842118108162707898890525894698160962108833920376795505059935112703645007973713125768662138084135228774544842466658822962605748361148457745081604549190156986296934965027718106862409382841715698056955265555204620839261388830826796349176704242669299463830842512564688372733269120922089406251010437634980936383704472101

In [28]:
import gmpy2
from gmpy2 import t_mod,t_div,invert,c_mod,iroot
import numpy as np

In [6]:
c_texts = np.array([c1,c2,c3])

In [20]:
remainders = np.array([t_mod(c_texts[i],pk_list[i][1]) for i in range(len(pk_list))])

In [21]:
N = pk_list[0][1]*pk_list[1][1]*pk_list[2][1]

In [22]:
Mi = np.array([t_div(N,pk_list[i][1]) for i in range(len(pk_list))])

In [23]:
Mi_1 =np.array([invert(Mi[i],pk_list[i][1]) for i in range(len(pk_list))])

In [30]:
C = t_mod(np.sum(c_texts*Mi*(Mi_1)),N)

In [31]:
C

mpz(224905097197692539041103720892798813176596535540077820958363031818142865029584005454193167330030352343178625056688897140663253587114677335365068018178587579275650613421083264678568070062647042424127536341712382562206411441811706183790782767912296100382954849428381986719989304816108777343222430082301404338158047730341783310132058033958358489456329615744771523555490710413354477692328466988845360429099183601832610254220540963753501879830420151693419250789623854736709236653556297985927774923732413905175235107647988959842982169472564761632460224717417272978099278333926610622983060488535789025987867953506463864796430701143486253367905868209911893511258481211798233552600427277768054881979364882207625032607231457072364988726234774913517724236019629814221667578913502899312772697958489365503261242262587246209277998947900997307166369341271200619752942046410396472373233717323401374369754113224645409769213236054328353969927050109353907626432893560989303477595941169572850321604339218384699199593

In [35]:
M = iroot(C,3)[0]

In [45]:
long_to_bytes(M)

b"Congratulations! Here is your flag: CRYPTOTRAINING{3_p30pl3_c4n7_k33p_4_s3cr3t}. Also some placeholder text, because I need it to be around 100 bytes to overflow the modulus, since I don't want to use paddings. YET"